In [1]:
import pandas as pd
# from herbie import Herbie
import xarray as xr
import os
import numpy as np
from pathlib import Path

In [8]:
# grid_daily_files
aifs_path = '/scratch2/aifs-data/tp/'
# os.walk(aifs_path)

In [14]:
flag = 'IFS'

all_files = os.listdir(aifs_path)
if flag=='AIFS':
    grib_daily_files = [os.path.join(aifs_path, file) for file in all_files if ('AIFS' in file) & (file[-5:]=='grib2')]
elif flag=='IFS':
    grib_daily_files = [os.path.join(aifs_path, file) for file in all_files if ('AIFS' not in file) & (file[-5:]=='grib2')]

In [15]:
grib_daily_files

['/scratch2/aifs-data/tp/IFS_2025-07-02-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-03-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-04-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-05-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-06-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-07-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-08-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-09-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-10-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-11-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-12-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-13-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-14-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-15-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-16-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-17-06_TP_FXX24.grib2',
 '/scratch2/aifs-data/tp/IFS_2025-07-18-

In [16]:
# Example chunking tuned for repeated Germany bbox over many days
chunks = {"time": 16, "latitude": 256, "longitude": 256}  # adjust if multi-level
if flag=='AIFS':
    ds = xr.open_mfdataset(sorted(grib_daily_files[:]), engine="cfgrib", combine="nested", concat_dim='time',
                       parallel=True, chunks=chunks)
elif flag=='IFS':
    ds = xr.open_mfdataset(sorted(grib_daily_files[:]), engine="cfgrib", combine="nested", concat_dim='time',
                       parallel=True, chunks=chunks, filter_by_keys={'dataType': 'pf'})

/tmp/ipykernel_4108115/2332230404.py:7: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'number' ('number',) The recommendation is to set join explicitly for this case.
  ds = xr.open_mfdataset(sorted(grib_daily_files[:]), engine="cfgrib", combine="nested", concat_dim='time',
/tmp/ipykernel_4108115/2332230404.py:7: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  ds = xr.open_mfdataset(sorted(grib_daily_files[:]), eng

In [17]:
# Germany extent
# ds_de = ds.sel(latitude=slice(55,47), longitude=slice(5,16))

ds_de = ds

In [18]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    if flag=='AIFS':
        ds_de.to_zarr("AIFS_TP_FXX24.zarr", mode="w", consolidated=True)
    elif flag=='IFS':
        ds_de.to_zarr("IFS_TP_FXX24.zarr", mode="w", consolidated=True)

/scratch2/mg963/conda_envs/envs/regrid/lib/python3.11/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


[########################################] | 100% Completed | 30m 5ss
